In [11]:
from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.behavior import Position, SpatialSeries
from pynwb.file import Subject
from uuid import uuid4
import yaml
from datetime import datetime
import os
from neuroconv.datainterfaces import OpenEphysRecordingInterface

# Create YAML file for metadata (Maybe using Frank Lab Rec-to-YAML?)

# Define some functions here for now

def initialize_nwb(metadata: dict):
    '''
    Constructs an NWBFile with basic session data

    Args:
    metadata (dict): basic metadata for this session

    Returns:
    NWBFile: nwb file with basic session information
    '''

    nwbfile = NWBFile(
        session_description=metadata["session_description"], # required
        identifier=str(uuid4()), # required, must be unique, need not be human readable
        session_start_time=metadata['session_start_time'], # required
        experimenter=metadata["experimenter"],
        lab=metadata["lab"],
        institution=metadata["institution"],
        session_id=metadata["session_id"],
        experiment_description=metadata["experiment_description"]
    )
    return nwbfile


def get_open_ephys_path(rat, date, directory='/Volumes/Tim/Ephys'):
    '''
    Attempt to find OpenEphys recording directory for the given rat and date.
    The assumed directory structure is {directory}/{rat}/{date}/etc
    Complain if no directory is found. 

    Args:
    rat (string): The ID of the rat
    date (string): The date to load, in MMDDYYYY format
    directory (string): The base directory to load from

    Returns:
    string: Path to the OpenEphys recording, or None if not found
    '''
    # Get the path to all data for this session (rat + date)
    ephys_session_path = f"{directory}/{rat}/{date}"
    # Check if the folder exists for this session
    if not os.path.exists(ephys_session_path):
        return f"The folder {ephys_session_path} does not exist."
    
    # Convert date string from MMDDYYYY to YYYY-MM-DD
    # because this is what the OpenEphys recording folder starts with
    date_string = datetime.strptime(date, "%m%d%Y").strftime("%Y-%m-%d")

    # Search for the OpenEphys recording folder starting with the date
    for folder_name in os.listdir(ephys_session_path):
        if folder_name.startswith(date_string):
            return f"{ephys_session_path}/{folder_name}"


In [12]:
# Path to YAML file for basic metadata
metadata_file_path = 'berke_lab_metadata.yml'

# Session info
rat = 'IM-1478'
date = '07192022'
nwb_name = f'{rat}_{date}.nwb'

# Read the metadata from the YAML file into a dictionary
with open(metadata_file_path, 'r') as file:
    metadata = yaml.safe_load(file)

# Create an NWB using this metadata
nwbfile = initialize_nwb(metadata)
print(nwbfile)

# Save it
with NWBHDF5IO(nwb_name, "w") as io:
    io.write(nwbfile)

/Users/steph/berkelab/jdb_to_nwb/venv/lib/python3.12/site-packages/pynwb/file.py:472: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


root pynwb.file.NWBFile at 0x5039848720
Fields:
  experiment_description: Hex maze task
  experimenter: ['Tim Krausz']
  file_create_date: [datetime.datetime(2024, 7, 11, 17, 12, 23, 850825, tzinfo=tzlocal())]
  identifier: 2d3bd02c-f613-4b17-839a-475fa55fd442
  institution: UCSF
  lab: Berke Lab
  session_description: Barrier change session with 3 blocks, ephys and photometry
  session_id: IM-1478_07192022
  session_start_time: 2022-07-19 00:00:00-07:00
  timestamps_reference_time: 2022-07-19 00:00:00-07:00



In [17]:
# Now add ephys stuff to that NWB!

# Get path to open ephys recording for the session
open_ephys_folder = get_open_ephys_path(rat, date)
print(open_ephys_folder)

# Read Open Ephys data for this session
interface = OpenEphysRecordingInterface(folder_path=open_ephys_folder)
# Extract what metadata we can from the source files
open_ephys_metadata = interface.get_metadata()
print(open_ephys_metadata)

# Open the existing NWB file in 'a' mode (append mode)
with NWBHDF5IO(nwb_name, 'a') as io:
    # Read the existing NWB file
    nwbfile = io.read()

    # Update session start time with exact time from Open Ephys
    nwbfile.session_start_time = open_ephys_metadata['NWBFile'].get('session_start_time', nwbfile.session_start_time)

    # Run the conversion to append data
    interface.run_conversion(
        nwbfile=nwbfile,
        metadata=open_ephys_metadata,
        save_to_file=False  # Modify the existing NWB instead of saving to new file
    )

    # Write the changes back to the NWB file
    io.write(nwbfile)

/Volumes/Tim/Ephys/IM-1478/07192022/2022-07-19_13-12-30
DeepDict: {'NWBFile': {'session_description': '', 'identifier': 'd5f7e30b-af4f-4613-a935-a7026b396e67', 'source_script': 'Created using NeuroConv v0.4.11', 'source_script_file_name': '/Users/steph/berkelab/jdb_to_nwb/venv/lib/python3.12/site-packages/neuroconv/basedatainterface.py', 'session_start_time': datetime.datetime(2022, 7, 19, 13, 12, 30)}, 'Ecephys': {'Device': [{'name': 'DeviceEcephys', 'description': 'no description'}], 'ElectrodeGroup': [{'name': 'ElectrodeGroup', 'description': 'no description', 'location': 'unknown', 'device': 'DeviceEcephys'}], 'ElectricalSeries': {'name': 'ElectricalSeries', 'description': 'Acquisition traces for the ElectricalSeries.'}}}


AttributeError: property of 'NWBFile' object has no setter

In [16]:
# Load the NWB to check it out

# Open the NWB file
with NWBHDF5IO(nwb_name, 'r') as io:
    nwbfile = io.read()

    # Print basic metadata
    print(f"Session description: {nwbfile.session_description}")
    print(f"Identifier: {nwbfile.identifier}")
    print(f"Session start time: {nwbfile.session_start_time}")
    print(f"Experimenter: {nwbfile.experimenter}")
    
    # List all available data
    print("\nAcquisition data:")
    for acquisition in nwbfile.acquisition:
        print(f" - {acquisition}")

    print("\nProcessing modules:")
    for module_name in nwbfile.processing:
        print(f" - {module_name}")

    print("\nStimulus presentations:")
    for stimulus in nwbfile.stimulus:
        print(f" - {stimulus}")
    
    # Inspect specific acquisition data (assuming there is at least one)
    if nwbfile.acquisition:
        acquisition_name = list(nwbfile.acquisition.keys())[0]
        acquisition_data = nwbfile.acquisition[acquisition_name]
        print(f"\nDetails of acquisition data '{acquisition_name}':")
        print(f" - Data type: {type(acquisition_data)}")
        print(f" - Description: {acquisition_data.description}")
        print(f" - Starting time: {acquisition_data.starting_time}")
        print(f" - Rate: {acquisition_data.rate}")
    
    # Inspect electrode data
    if nwbfile.electrodes is not None:
        print("\nElectrodes table:")
        print(nwbfile.electrodes.to_dataframe())


Session description: Barrier change session with 3 blocks, ephys and photometry
Identifier: 2d3bd02c-f613-4b17-839a-475fa55fd442
Session start time: 2022-07-19 00:00:00-07:00
Experimenter: ('Tim Krausz',)

Acquisition data:

Processing modules:

Stimulus presentations:
